# Calculate fluxes of major ions into crust

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%run figures.py

Returned 24 files without errors out of a total possible 24.
0 files had errors.
0 files had unhandled errors.
File failure rate: 0.0 %.
To see unhandled errors, run with verbose=True.


In [4]:
import numpy as np

time_steps = 25
i = 0
file_num=0
time = 20
burn_end_t = 3

category = 'volume'
plot_var = 'Calcite'

scale_factors = (1,1,1,1)
range_sets = [(0,5), (6,9), (10,14), (15,19)]

dataset = set1

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
anhyd_mv = 46.010
calcite_mv = 36.934
vf_to_mols = cell_vol / anhyd_mv
vf_to_mols_calc = cell_vol / calcite_mv
scale= vf_to_mols_calc

iterables = zip(np.arange(len(range_sets)), range_sets)
# Axis level loop
calc_list = ['File Number', 'Net moles', 'Net moles after warmup', 'Percentage moles after warmup', 'Flux']
calc_dict = {calc: [] for calc in calc_list}
for j, range_set in enumerate(range_sets):
    index = np.arange(range_set[0], range_set[1]+1)

    for ind in index:
        try:
            result = dataset[ind].results[category]
            burn_end = result.isel(time=burn_end_t, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
            val = result.isel(time=time, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
            net = val - burn_end
            percentage_after_burn = net / val
            duration = (result.time[time] - result.time[burn_end_t]).to_numpy()
            flux = net / duration

            calc_results = [ind, val, net, percentage_after_burn, flux]

            for calc, calc_result in zip(calc_list, calc_results):
                calc_dict[calc].append(calc_result)
        except IndexError:
            calc_results = [ind, np.NaN, np.NaN, np.NaN, np.NaN]
            for calc, calc_result in zip(calc_list, calc_results):
                calc_dict[calc].append(calc_result)


In [73]:
import pandas as pd
df = pd.DataFrame.from_dict(calc_dict)
df = df.set_index('File Number')
df.to_csv('output/flux_data.csv')